## Objectives
The purpose of this notebook is to gather, load, and perform a preliminary inspection of the raw data for the Heritage Housing project. This step ensures we're working with valid data sources, understand their structure, and prepare for cleaning and wrangling in the next phase.


## Source Files
- Kaggle JSON Authentication Token: Required to access the Kaggle API.

- We are using three key files from the Kaggle dataset:

  - house_prices_records.csv: Historical house sale records.

  - inherited_houses.csv: List of inherited houses we want to estimate prices for.

  - house-metadata.txt: Describes the features present in both CSVs.

    These were placed in the /data folder for structured access.

### Change Working Directory

- 
  To allow smooth access to the data files, we need to adjust our working directory. Since this notebook lives in a subfolder (e.g. jupyter_notebooks), we’ll move one level up to the project root.

In [8]:
import os

# Display the current directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# Move to the parent directory
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

# Confirm the change
current_dir = os.getcwd()
print(f"New directory: {current_dir}")

Current directory: /workspaces/heritage_housing/jupyter_notebooks
You set a new current directory
New directory: /workspaces/heritage_housing


### Fetch Dataset from Kaggle

- To keep the workflow reproducible and professional, we will use Kaggle’s API to programmatically download the dataset.

Setup: Install Kaggle and Authenticate

In [9]:
# Install Kaggle CLI (if not already installed)
!pip3 install kaggle==1.5.12

  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73026 sha256=e6b97c9f4058ea4c5d8989bfaf3288727a6d55898d93a95f9bbf7e65371f76a4
  Stored in directory: /home/cistudent/.cache/pip/wheels/f5/69/4d/d701fc604b9fb09be59718b4056fd5556a22588ce1f25dd090
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.7.4.2
    Uninstalling kaggle-1.7.4.2:
      Successfully uninstalled kaggle-1.7.4.2

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


- You must first create a Kaggle account and generate an API token from your account settings. This will download a kaggle.json file.
- Move kaggle.json to the root directory of this repo.
- Run the below to register the token and adjust permissions:

In [10]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
!chmod 600 kaggle.json  # Secure the file

### Download the Dataset

- We now fetch the dataset using the CLI. This project uses the "Heritage Housing Predictor" dataset from Kaggle competitions.

In [11]:
KaggleDatasetPath = "competitions/heritage-housing-predictor"
DestinationFolder = "data"

!kaggle {KaggleDatasetPath} download -c heritage-housing-predictor -p {DestinationFolder}

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: argument command: invalid choice: 'competitions/heritage-housing-predictor' (choose from 'competitions', 'c', 'datasets', 'd', 'kernels', 'k', 'config')


- Unzip the Downloaded File

In [12]:
import zipfile
from pathlib import Path

for zip_file in Path(DestinationFolder).glob("*.zip"):
    with zipfile.ZipFile(zip_file, 'r') as z:
        z.extractall(DestinationFolder)
    zip_file.unlink()  # Delete zip file after extraction

### Load Required Libraries

In [13]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
pd.options.display.max_rows = None

### Load Datasets

In [20]:
# Load house sales records
house_prices_df = pd.read_csv('data/house_prices_records.csv')

# Load inherited houses data
inherited_df = pd.read_csv('data/inherited_houses.csv')

### Quick Peek at Each Dataset
- This gives us a rough idea of the data shape and the kind of features we’ll be dealing with.

In [21]:
print("\n--- House Prices Data ---")
print(house_prices_df.shape)
print(house_prices_df.columns)
house_prices_df.head()

print("\n--- Inherited Houses Data ---")
print(inherited_df.shape)
print(inherited_df.columns)
inherited_df.head()


--- House Prices Data ---
(1460, 24)
Index(['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr', 'BsmtExposure', 'BsmtFinSF1',
       'BsmtFinType1', 'BsmtUnfSF', 'EnclosedPorch', 'GarageArea',
       'GarageFinish', 'GarageYrBlt', 'GrLivArea', 'KitchenQual', 'LotArea',
       'LotFrontage', 'MasVnrArea', 'OpenPorchSF', 'OverallCond',
       'OverallQual', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt', 'YearRemodAdd',
       'SalePrice'],
      dtype='object')

--- Inherited Houses Data ---
(4, 23)
Index(['1stFlrSF', '2ndFlrSF', 'BedroomAbvGr', 'BsmtExposure', 'BsmtFinSF1',
       'BsmtFinType1', 'BsmtUnfSF', 'EnclosedPorch', 'GarageArea',
       'GarageFinish', 'GarageYrBlt', 'GrLivArea', 'KitchenQual', 'LotArea',
       'LotFrontage', 'MasVnrArea', 'OpenPorchSF', 'OverallCond',
       'OverallQual', 'TotalBsmtSF', 'WoodDeckSF', 'YearBuilt',
       'YearRemodAdd'],
      dtype='object')


,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,EnclosedPorch,GarageArea,GarageFinish,GarageYrBlt,GrLivArea,KitchenQual,LotArea,LotFrontage,MasVnrArea,OpenPorchSF,OverallCond,OverallQual,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd
0,896,0,2,No,468.0,Rec,270.0,0,730.0,Unf,1961.0,896,TA,11622,80.0,0.0,0,6,5,882.0,140,1961,1961
1,1329,0,3,No,923.0,ALQ,406.0,0,312.0,Unf,1958.0,1329,Gd,14267,81.0,108.0,36,6,6,1329.0,393,1958,1958
2,928,701,3,No,791.0,GLQ,137.0,0,482.0,Fin,1997.0,1629,TA,13830,74.0,0.0,34,5,5,928.0,212,1997,1998
3,926,678,3,No,602.0,GLQ,324.0,0,470.0,Fin,1998.0,1604,Gd,9978,78.0,20.0,36,6,6,926.0,360,1998,1998


### Initial Observations

In this section, make notes like:

- Are there null values?

- Do both datasets share a common ID column for merging?

- Are there categorical or numerical inconsistencies?

- What’s the size of the datasets?

## Our Initial Observations

- house_prices_records.csv contains 1,460 rows and 24 columns. The target variable SalePrice is included.

- inherited_houses.csv contains 4 rows and 23 columns — all the same features except for the missing SalePrice column.

- No explicit ID column is shared between the datasets, so merging isn’t directly possible.

- Features include both numeric (e.g., LotArea, YearBuilt) and categorical data (e.g., BsmtExposure, KitchenQual).

- Likely nulls in columns like GarageYrBlt, LotFrontage, and basement features.

- Column names match well between datasets — suggesting they're aligned in structure.

## Summary of Actions Completed

- Changed working directory to project root.

- Installed Kaggle CLI and configured authentication.

- Programmatically downloaded and extracted all raw data from Kaggle.

- Loaded raw datasets and confirmed structural consistency.

- Inspected column names, dimensions, and got a feel for data types and formats.

- Noted down initial findings to guide the cleaning strategy.

## Next Steps

Proceed to 02_Data_Cleaning.ipynb, where we will:

- Handle missing values.

- Remove duplicates or inconsistent entries.

- Normalize categorical fields.

- Possibly merge the two datasets, if needed.